## Création des infos customers

### Source : https://unprenom.fr/fille/categories:48 
### Source 1 : https://onomastique.hypotheses.org/458 other source https://studentsoftheworld.info/penpals/stats_fr.php?Pays=SEN
### Source 2 : https://fr.geneawiki.com/wiki/Noms_de_famille_s%C3%A9n%C3%A9galais

In [1]:
# importation de package necessaire
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import bcrypt
import random
import string
from unidecode import unidecode
import warnings 
warnings.filterwarnings("ignore")

### Chargement du dataset dim_geographies

In [2]:
df_geography = pd.read_csv("dataset/geographies.csv")
df_geography

,Geo_Key,Country,Region,Department,District,Creation_date
0,1,Sénégal,Dakar,Dakar,Gueule Tapee Fass Colobane,2022-03-01
1,2,Sénégal,Dakar,Dakar,Fann Point E Amitie,2022-03-01
2,3,Sénégal,Dakar,Dakar,Goree,2022-03-01
3,4,Sénégal,Dakar,Dakar,Medina,2022-03-01
4,5,Sénégal,Dakar,Dakar,Plateau,2022-03-01
...,...,...,...,...,...,...
426,427,Sénégal,Kédougou,Saraya,Bembou,2022-03-01
427,428,Sénégal,Kédougou,Saraya,Khossanto,2022-03-01
428,429,Sénégal,Kédougou,Saraya,Medina Baffe,2022-03-01
429,430,Sénégal,Kédougou,Saraya,Missirah Sirimana,2022-03-01


## Définition des fonctions que nous utiliserons dans la génération de nos customers

In [3]:
def email_generation(first_name, surname):
    number=''
    first_name = unidecode(first_name.replace(" ","").lower())
    surname = unidecode(surname.replace(" ","").lower())
    with_digit=random.choice('012')
    if(with_digit == '0') :
        number = ''
    if(with_digit == '1') : 
        number = random.choice("0123456789")
    if(with_digit == '2') : 
        number = random.choice("0123456789") 
        number=number+random.choice("0123456789") 
        
    email = first_name+surname+number+"@gmail.com"
    return email

def random_number(number_of_digit):
    number=''
    for i in range(number_of_digit):
        number = number + random.choice("0123456789")
    return number

def phone_number_generation():
    phone_number = "+221" + random.choice(["77","78"]) + random_number(3) + random_number(2) + random_number(2)  
    return phone_number

def password_generation():
    letters = string.ascii_letters + 'éèàâç'
    digit_punctuation = string.digits + "!.#@?;"
    # Generate first letter
    first_letter = random.choice(string.ascii_letters)
    # Generate remaining characters
    remaining_letter = ''.join(random.choices(letters + digit_punctuation, k=random.choice([8,9,10,11,12,13,14]) ))
    password = first_letter + remaining_letter 
    hashed_password = bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt())
    
    return hashed_password

In [4]:
def creation_cust(first_name, surname, nbre, sexe):
    first_name = list(map(str.capitalize, first_name))
    surname = list(map(str.capitalize, surname))
    lst_full_name = []
    lst_full_cust=[]
    while (len(lst_full_name) <nbre):
        first_name_random = random.choice(first_name)
        surname_random = random.choice(surname)
        full_name = first_name_random+" "+surname_random
        if(full_name not in lst_full_name):
            dic_cust = {}
            lst_full_name.append(full_name)
            dic_cust["Cust_firstname"] = first_name_random
            dic_cust["Cust_lastname"] = surname_random
            dic_cust["Cust_email"] = email_generation(first_name_random, surname_random)
            dic_cust["Cust_phone_number"] = phone_number_generation()
            dic_cust["Cust_password"] = password_generation()
            dic_cust["Cust_sexe"] = sexe
            lst_full_cust.append(dic_cust)
    return lst_full_cust

In [5]:
### génération aléatoire d'un en 
def random_geography_cust(df_geography, region) :
    geography_department = df_geography[df_geography["Region"] == region]
    number_random = random.randint(0, len(geography_department)-1)
    random_geography_cust = geography_department.iloc[number_random]
    return random_geography_cust

In [6]:
def customers_geographies(df_temp, df_geography, region, indice_deb, indice_end) :
    df_geo_region = df_temp.iloc[indice_deb:indice_end]
    for index, row in df_geo_region.iterrows():
        geography_random = random_geography_cust(df_geography, region)
        row["Cust_region"] = geography_random[2]
        row["Cust_department"] = geography_random[3]
        row["Cust_district"] = geography_random[4]
    return df_geo_region


In [7]:
def generate_random_dates(start_date, end_date, count):
    # conversion des dates en objet datetime
    if not isinstance(start_date, datetime):
        start_date = datetime.strptime(start_date, "%Y-%m-%d")
    if not isinstance(end_date, datetime):
        end_date = datetime.strptime(end_date, "%Y-%m-%d")
    
    # nous allons calculer le nombre de jour entre les deux dates
    delta = end_date - start_date

    random_dates = [] # création de la liste qui va contenir les dates que nous allons générer
    for _ in range(count):
        # maintenant nous allons générer un nombre alétoire entre 0 et delta
        random_days = random.randint(0, delta.days)
        # ajoutons le nombre générer à la date de début pour obtenir une date aléatoire
        random_date = start_date + timedelta(days=random_days)
        random_date = random_date.strftime("%Y-%m-%d")
        random_dates.append(random_date)

    return random_dates

In [8]:
# Création de liste de prénoms féminin et masculin sénégalais
# Création de liste de noms sénégalais

first_name_femal = ["Fatou", "Léa", "Banel", "Aminata", "Aichetou", "Mireille", "Chantal", "Gisele", "Maya","Daya", "Nafissa", "Patricia", "Cécilia", "Djeneba", "Nina", "Doua", "Kora", "Awa", "Khady", "Aissatou", "SEYNABOU", "MARIAMA", "MAIMOUNA", "ROKHAYA", "FATOUMATA", "COUMBA", "NDEYE FATOU", "ADAMA", "ASTOU", "KHADIDIATOU", "MARIEME", "AMY", "DIEYNABA", "AIDA", "RAMATOULAYE", "JOE", "Thérèse", "Madelaine", "Marie", "Yaya","Olivia", "Mary", "Gina", "Dija", "Amy", "Paulette", "Anna", "Josephine", "Rose", "Elisabeth", "Sabrina", "Hélène", "Sophie", "Sara", "Irène", "Mariam", "Aicha", "Aida", "Fatima", "Fatoumata", "Maty", "Hawa", "Issa", "Aissata", "Kady", "Bintou", "Fatim", "Néné", "Penda", "Ramata", "Diarra", "Rama", "Ndeye Bourry", "Filassira", "Ndeye Khady" "Kadiatou", "Salimata", "Denisia", "Oulimata", "Arame", "Rokhaya", "Madeleine", "Anne", "Soda", "Daba", "Marcella", "Yama", "Louise", "Dominique", "Fatouma", "Odette", "Charlotte", "Béatrice", "Amara", "Isabelle", "Ramatoulaye", "Fama", "Nogaye", "Binta","Diarrie", "Dieynaba", "Halimatou", "Amina", "Lala", "Khadija", "Mabintou", "Nafissatou"]
first_name_male = ["MAMADOU", "IBRAHIMA", "ABDOULAYE", "MOUSSA", "AMADOU", "BABACAR", "OUMAR", "CHEIKH", "MOUSTAPHA", "ALIOU", "ASSANE", "ABDOU", "ADAMA", "ALIOUNE", "SOULEYMANE", "MBAYE", "MALICK", "LAMINE", "SAMBA", "Thierno"]
surname = ["Diop", "Diallo", "Niang", "Mbaye", "Gueye", "Cisse", "Faye", "Sow", "Ba", "Dia", "Diagne", "Dieng", "Toure", "Camara", "Drame", "Diedhiou", "Dieye", "Mendy", "Mballo", "Badiane", "Sagna", "Seydi", "Cissokho", "Sambou", "Mbow", "Sonko", "Mbacke", "Boye", "Goudiaby", "Gassama", "Manga", "Faty", "Gningue", "Niasse", "Bodian", "Aidara", "Kamara","Diadhiou", "Ly", "Ndour", "Diamanka", "Sam", "Biagui", "Seck", "Thiare", "Sarr", "Cissoko", "Sock", "Ndom", "Nguirane", "Ndior", "Boubane", "Oualy", "Massaly", "Beye", "Badji", "Guisse", "Yade", "Danfa", "Diawo", "Diouck", "Fedior"]

# génération 560 customers de sexe féminin (first_name, last_name, email, phone_number, password)
list_cust_female = creation_cust(first_name_femal, surname, 560, "Féminin")
# génération 80 customers de sexe masculin (first_name, last_name, email, phone_number, password) 
list_cust_male = creation_cust(first_name_male, surname, 80, "Masculin")

# chargement de ces customers généré dans un dataframe temporaire 
lst_columns = ["Cust_ID", "Cust_firstname", "Cust_lastname", "Cust_email", "Cust_phone_number", "Cust_password", "Cust_sexe","Cust_country", "Cust_region", "Cust_department", "Cust_district", "Cust_date_register"]
df_temp = pd.DataFrame(columns = lst_columns )
for c in list_cust_female : 
    df_temp = df_temp.append(c, ignore_index=True)
for c in list_cust_male : 
    df_temp = df_temp.append(c, ignore_index=True)
    
# interessons nous maintenant à la génération des données ( géographique pour les clients générés)
# Création du df_customers ( df avec tous les champs)

lst_columns = ["Cust_ID", "Cust_firstname", "Cust_lastname", "Cust_email", "Cust_phone_number", "Cust_password", "Cust_sexe", "Cust_country", "Cust_region", "Cust_department", "Cust_district", "Cust_date_register"]
df_customers = pd.DataFrame(columns = lst_columns )

# génération des données pour les colonnes Cust_region, Cust_department, Cust_district
dic_geo_repartition = {"Repartition_1": ["Dakar", 0 , 240], "Repartition_2": ["Thiès", 240 , 420], "Repartition_3": ["Ziguinchor", 420 , 480], 
               "Repartition_4": ["Kaolack", 480 , 560], "Repartition_5": ["Dakar", 560 , 620], "Repartition_6": ["Thiès", 620 , 640]}

for repartion in dic_geo_repartition:
    info_repartion = dic_geo_repartition[repartion]
    df_cust_temp = customers_geographies(df_temp, df_geography, info_repartion[0], info_repartion[1], info_repartion[2])
    df_customers = df_customers.append(df_cust_temp, ignore_index=True)


# génération des données pour la colonne Cust_date_register
dic_repartition_date = {"1" : ["2022-03-20", "2022-07-31", 100],
 "2" : ["2022-07-31", "2022-09-30", 50],
 "3" : ["2022-09-30", "2022-11-30", 40],
 "4" : ["2022-11-30", "2023-01-12", 100],
 "5" : ["2023-01-12", "2023-03-31", 40],
 "6" : ["2023-03-31", "2023-05-31", 120],
 "7" : ["2023-05-31", "2023-11-30", 90],
 "8" : ["2023-11-30", "2024-02-11", 80],
 "9" : ["2024-02-11", "2024-03-30", 20]
}
random_dates = []
for i in dic_repartition_date :
    info = dic_repartition_date[i]
    random_dates.extend(generate_random_dates(info[0], info[1], info[2]))

df_customers = df_customers.sample(frac=1).reset_index(drop=True) 
df_customers["Cust_date_register"] = random_dates
df_customers["Cust_country"] = ["Sénégal"]*640

# Ajout des données pour la colonne Cust_ID
list_Cust_ID = []
for i in range(20, len(df_customers)+20):
    Cust_ID = f"A{i:06d}"
    list_Cust_ID.append(Cust_ID)

df_customers = df_customers.sort_values(by='Cust_date_register', ascending=True)
df_customers["Cust_ID"] = list_Cust_ID

In [9]:
df_customers.head(5)

,Cust_ID,Cust_firstname,Cust_lastname,Cust_email,Cust_phone_number,Cust_password,Cust_sexe,Cust_country,Cust_region,Cust_department,Cust_district,Cust_date_register
44,A000020,Fama,Dieye,famadieye81@gmail.com,+221787983005,b'$2b$12$9WcEQOPqZa3F25uXGRFqLuRXywacefdp0Uqo6...,Féminin,Sénégal,Thiès,Thiès,Ngoundiane,2022-03-20
92,A000021,Aicha,Sagna,aichasagna6@gmail.com,+221774160499,b'$2b$12$uGFf3PlJtaFSvc.WBRn65uCZmU/c5oiatpaop...,Féminin,Sénégal,Kaolack,Kaolack,Ndiaffate,2022-03-20
16,A000022,Abdou,Diamanka,abdoudiamanka@gmail.com,+221771545961,b'$2b$12$veizFY0Nsg7lySk3pjUeAOr0tgomFoZ9oqtiH...,Masculin,Sénégal,Dakar,Dakar,Hlm,2022-03-22
20,A000023,Abdoulaye,Sam,abdoulayesam92@gmail.com,+221776361782,b'$2b$12$WHsalQG4w5ZG15pwmIBrxufIqgGqX2wv/VAsm...,Masculin,Sénégal,Dakar,Guédiawaye,Ndiareme,2022-03-23
24,A000024,Lamine,Niang,lamineniang@gmail.com,+221786288713,b'$2b$12$M28DMsaXx.iEh45VW/h7x.WZ9rHPpRs8kNpLw...,Masculin,Sénégal,Dakar,Guédiawaye,Medina Gounass,2022-03-23


### sauvegarde dans un fichier csv

In [10]:
df_customers.to_csv("dataset/customers.csv",encoding='utf-8-sig', index=False)

In [11]:
len(df_customers)

640